In [1]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
from rdflib import Graph
from pysolr import Solr

In [13]:
from api.src.function.cataloguing.generate_id import GenerateId

In [14]:
work_id = GenerateId()
work_id['id']

'bk-6'

SOLR

In [2]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">424</int>\n</lst>\n</response>\n'

In [3]:
solrAcervo = Solr('http://localhost:8983/solr/acervo/', timeout=10)
solrAcervo.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">239</int>\n</lst>\n</response>\n'

In [11]:
doc = {
        "id": "n78078534",
        #"contributorTo": {"add": ["bk-1"]}
     }
solrAuthorities.add([doc], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":310}}\n'

In [ ]:
work = {
        "id": "request.instanceOf",
         "hasInstance": {"add": [request.instanceOf]}
    }

In [ ]:
solr = Solr('http://localhost:8983/solr/acervo/', timeout=10)
solr.ping()

DELETE GRAPHS

In [4]:
upAcervo = FusekiUpdate('http://localhost:3030', 'acervo')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upAcervo.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [5]:
upThesaurus = FusekiUpdate('http://localhost:3030', 'thesaurus')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upThesaurus.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [10]:
q = """SELECT ?g  
        {graph ?g {?s ?p ?o}} 
        group by ?g 
        ORDER BY DESC(?g)
        LIMIT 1"""

response = queryAcervo.run_sparql(q)
r = response.convert()
bindings = r['results']['bindings']

In [15]:
value = bindings[0]['g']['value']

In [21]:
value.split("-")[1]

'1'

In [ ]:
def GenerateId():

    year = datetime.now().year 
    year = str(year)[2:]

    # store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/acervo/update')
    # query_endpoint = 'http://localhost:3030/acervo/query'
    # update_endpoint = 'http://localhost:3030/acervo/update'
    # store.open((query_endpoint, update_endpoint))
    
    
    q = """SELECT ?g  
        {graph ?g {?s ?p ?o}} 
        group by ?g 
        ORDER BY DESC(?g)
        LIMIT 1"""
    
    # r = store.query(q)
    response = queryAcervo.run_sparql(d)
    
    bindings = r.bindings
    if len(bindings) == 0:
        register = f'bk-{year}-1'
        return register

    for i in r:
        uri = i[0].toPython()
    bk = uri.split('/')[-1]
    count = bk.split('-')[-1]
    count = int(count) + 1  

    
    register = f'bk-{year}-{count}'

    return register

WORK BK

In [5]:
uri = "http://localhost:3030/acervo?graph=https://bibliokeia.com/resources/work/bk-19"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('bk-19.nt')

<Graph identifier=Ndcc3a04a8b434776ac7e60679ded51b7 (<class 'rdflib.graph.Graph'>)>

WORK LOC

In [2]:
g = Graph()
g.parse("http://id.loc.gov/resources/works/18334774.bibframe.rdf")
g.serialize('glotious_forest.nt')

<Graph identifier=Nb0c63005ff254156a92d75a5355da141 (<class 'rdflib.graph.Graph'>)>

INSTANCE BK

In [17]:
uri = "http://localhost:3030/acervo?graph=https://bibliokeia.com/bibframe/instance/bk-23-1"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('instance.nt')

<Graph identifier=Nef0292801dbc407b9c650620a490d46b (<class 'rdflib.graph.Graph'>)>

ITEMS BK

In [15]:
uri = 'http://id.loc.gov/resources/items/2215252-050-9.rdf'
g = Graph()
g.parse(uri)
g.serialize('item.nt')

<Graph identifier=N81349e2591274bada155170aeee81ea7 (<class 'rdflib.graph.Graph'>)>

NAMES BK

In [9]:
uri = "http://localhost:3030/thesaurus?graph=https://bibliokeia.com/authorities/names/n79138841"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('nameBK.nt')

<Graph identifier=N67ae810991d64c6789163725f3ea993c (<class 'rdflib.graph.Graph'>)>

In [24]:
token = "n79138841"

qType = """
PREFIX namesBK: <https://bibliokeia.com/authorities/names/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
SELECT  ?o
WHERE { namesBK:"""+token+" rdf:type ?o}"

r = g.query(qType)

for i in r:
    tipo = i.o.split("#")[1]
    if tipo != "Authority":
        print(tipo)



CorporateName


SUBJECTS BK

In [8]:
token = "sh85084414"
uri = f'http://localhost:3030/thesaurus?graph=https://bibliokeia.com/authorities/subjects/{token}'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('subjects.nt')

<Graph identifier=N8687b45b63cc46f8bed88bf7903542c0 (<class 'rdflib.graph.Graph'>)>

SERIE LOC

In [7]:
uri = "http://id.loc.gov/resources/hubs/3035b85d-c7b5-649a-aa12-05ce677f9f11.bibframe.rdf"
g = Graph()
g.parse(uri)
g.serialize('serie.nt')

<Graph identifier=N94c42584427b4fe1af17b398a24ba0d3 (<class 'rdflib.graph.Graph'>)>

SERIE BK

In [4]:

uri = f'http://localhost:3030/acervo?graph=https://bibliokeia.com/resources/hub/bk-5'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('serieBK.nt')

<Graph identifier=Ned294ddf7d07475cb079fe57c21f7fa5 (<class 'rdflib.graph.Graph'>)>

NAMES LOC

In [7]:
uri = "http://id.loc.gov/authorities/names/n79138841.madsrdf.rdf"
g = Graph()
g.parse(uri)
g.serialize('names.nt')

<Graph identifier=Naaef8cb88add48bbb714a2c129becfcb (<class 'rdflib.graph.Graph'>)>

In [7]:
from api.src.function.bibframe.types import TypeBF

In [1]:
from api.src.schemas.bibframe.hub import Hub_Schema

In [2]:
h = Hub_Schema(**{
    "type": "Series",
    "mainTitle": "Memoirs of the New York Botanical Garden",
    "subtitle": "",
    "variantTitle": "NYBG memoirs",
    "contributionAgent": "New York Botanical Garden",
    "contributionRole": "Contributor",
    "contributionRoleUri": "http://id.loc.gov/vocabulary/relators/ctb",
    "contributionID": "n79138841",
    "cdd": "548.2",
    "cutter": "",
    "issn": "0077-8931"
})

In [5]:
h.cutter == ""

True